In [1]:
import os
import sys
py_files_dir = os.path.abspath("/home/rebeccaz/Programs/VAMPIRES/vampires_calibration/vampires_on_sky_calibration/programs/py_files/")
sys.path.insert(0, py_files_dir)
sys.path.insert(0, "/home/rebeccaz/Programs/VAMPIRES/vampires_calibration/vampires_internal_dpp/programs/other/")

import em_gain
import numpy as np
import ast
import pandas as pd
import helper_functions as funcs

# Examing 09/19/2023 Data - Broadband with Error Propagation (final std)

In [2]:
# Define wavelengths
wavelengths = np.array(["Ha-Cont", "Halpha", "SII", "SII-Cont"])

# Read in CSV file
df = pd.read_csv("/home/rebeccaz/Data/20230919_VAMPIRES_Internal_Cal/collapsed/2023_09_19_Collapsed_Data.csv")

for i, wavelength in enumerate(wavelengths):
    print("Wavelength: " + str(wavelength))
    # Filter the DataFrame for the specific wavelength
    df_wavelength = df[(df["FILTER02"] == wavelength) & (df["OBS-MOD"] != "IPOL_MBI")]
    
    # Further filter for each camera from the already filtered DataFrame
    df_cam1_FL1 = df_wavelength[(df_wavelength["U_CAMERA"] == 1) & (df_wavelength["U_FLC"] == "A")]
    df_cam1_FL2 = df_wavelength[(df_wavelength["U_CAMERA"] == 1) & (df_wavelength["U_FLC"] == "B")]
    df_cam2_FL1 = df_wavelength[(df_wavelength["U_CAMERA"] == 2) & (df_wavelength["U_FLC"] == "A")]
    df_cam2_FL2 = df_wavelength[(df_wavelength["U_CAMERA"] == 2) & (df_wavelength["U_FLC"] == "B")]

    # Extracting fluxes for each camera & FLC state
    FL1 = df_cam1_FL1["TOTAL_FLUX"].values.astype(float)
    FL2 = df_cam1_FL2["TOTAL_FLUX"].values.astype(float)
    FR1 = df_cam2_FL1["TOTAL_FLUX"].values.astype(float)
    FR2 = df_cam2_FL2["TOTAL_FLUX"].values.astype(float)

    FL1_std = FL1.std()
    FL2_std = FL2.std()
    FR1_std = FR1.std()
    FR2_std = FR2.std()

    # print("FL1: " + str(FL1))
    # print("FL2: " + str(FL2))
    # print("FR1: " + str(FR1))
    # print("FR2: " + str(FR2))

    # Assuming em_gain.calculate_em_gain_ratio returns normalized_fluxes and em_gain_ratios
    normalized_fluxes, em_gain_ratios = \
        em_gain.calculate_em_gain_ratio(FL1, FL2, FR1, FR2)

    print("Number of Data Points: " + str(len(normalized_fluxes)))
    print(f"Median Normalized Flux for {wavelength}: {np.median(normalized_fluxes):.6f} ± {np.std(normalized_fluxes, ddof=1):.6f}")
    print(f"Median EM Gain Ratio for {wavelength}: {np.median(em_gain_ratios):.6f} ± {np.std(em_gain_ratios, ddof=1):.6f}")
    print()

Wavelength: Ha-Cont
Number of Data Points: 80
Median Normalized Flux for Ha-Cont: -0.073556 ± 0.062565
Median EM Gain Ratio for Ha-Cont: 1.158812 ± 0.144623

Wavelength: Halpha
Number of Data Points: 80
Median Normalized Flux for Halpha: -0.105422 ± 0.057540
Median EM Gain Ratio for Halpha: 1.235691 ± 0.141803

Wavelength: SII
Number of Data Points: 80
Median Normalized Flux for SII: -0.092415 ± 0.044353
Median EM Gain Ratio for SII: 1.203652 ± 0.106290

Wavelength: SII-Cont
Number of Data Points: 80
Median Normalized Flux for SII-Cont: -0.077267 ± 0.040790
Median EM Gain Ratio for SII-Cont: 1.167474 ± 0.096618



# Examing 09/14/2023 Data - Broadband with Error Propagation (Std of individual fluxes)

In [4]:
import numpy as np
import pandas as pd
from em_gain import calculate_em_gain_ratio_with_errors  # Adjust this import as needed

# Define wavelengths
wavelengths = np.array(["625-50", "675-50", "725-50", "750-50", "775-50"])

# Read in CSV file
df = pd.read_csv("/home/rebeccaz/Data/collapsed/2023_09_14_Collapsed_Data.csv")

for wavelength in wavelengths:
    print("Wavelength: " + str(wavelength))

    # Filter the DataFrame for the specific wavelength
    df_wavelength = df[(df["U_FILTER"] == wavelength) & (df["OBS-MOD"] != "IPOL_MBI")]

    # Subselect by camera and FLC state
    df_cam1_FL1 = df_wavelength[(df_wavelength["U_CAMERA"] == 1) & (df_wavelength["U_FLC"] == "A")]
    df_cam1_FL2 = df_wavelength[(df_wavelength["U_CAMERA"] == 1) & (df_wavelength["U_FLC"] == "B")]
    df_cam2_FL1 = df_wavelength[(df_wavelength["U_CAMERA"] == 2) & (df_wavelength["U_FLC"] == "A")]
    df_cam2_FL2 = df_wavelength[(df_wavelength["U_CAMERA"] == 2) & (df_wavelength["U_FLC"] == "B")]

    # Extract flux arrays
    FL1 = df_cam1_FL1["TOTAL_FLUX"].values.astype(float)
    FL2 = df_cam1_FL2["TOTAL_FLUX"].values.astype(float)
    FR1 = df_cam2_FL1["TOTAL_FLUX"].values.astype(float)
    FR2 = df_cam2_FL2["TOTAL_FLUX"].values.astype(float)

    # Truncate to smallest length to avoid index mismatches
    N = min(len(FL1), len(FL2), len(FR1), len(FR2))
    if N == 0:
        print("Insufficient data for wavelength:", wavelength)
        continue
    FL1, FL2, FR1, FR2 = FL1[:N], FL2[:N], FR1[:N], FR2[:N]

    # Calculate per-stream means and standard deviations
    FL1_mean, FL1_std = np.mean(FL1), np.std(FL1, ddof=1)
    FL2_mean, FL2_std = np.mean(FL2), np.std(FL2, ddof=1)
    FR1_mean, FR1_std = np.mean(FR1), np.std(FR1, ddof=1)
    FR2_mean, FR2_std = np.mean(FR2), np.std(FR2, ddof=1)

    # Call error-propagating gain ratio function
    normalized_flux, sigma_normalized_flux, em_gain_ratio, sigma_em_gain_ratio = calculate_em_gain_ratio_with_errors(
        FL1_mean, FL2_mean, FR1_mean, FR2_mean,
        FL1_std, FL2_std, FR1_std, FR2_std
    )

    # Print results
    print(f"Number of Data Points: {N}")
    print(f"Normalized Flux: {normalized_flux:.6f} ± {sigma_normalized_flux:.6f}")
    print(f"EM Gain Ratio:   {em_gain_ratio:.6f} ± {sigma_em_gain_ratio:.6f}")
    print()


Wavelength: 625-50
Number of Data Points: 80
Normalized Flux: -0.064823 ± 0.260154
EM Gain Ratio:   1.138632 ± 0.594940

Wavelength: 675-50
Number of Data Points: 80
Normalized Flux: -0.083230 ± 0.242013
EM Gain Ratio:   1.181573 ± 0.575901

Wavelength: 725-50
Number of Data Points: 80
Normalized Flux: -0.087077 ± 0.300004
EM Gain Ratio:   1.190766 ± 0.719927

Wavelength: 750-50
Number of Data Points: 80
Normalized Flux: -0.086409 ± 0.303634
EM Gain Ratio:   1.189164 ± 0.727573

Wavelength: 775-50
Number of Data Points: 80
Normalized Flux: -0.086448 ± 0.301243
EM Gain Ratio:   1.189256 ± 0.721906



# Examing 09/14/2023 Data - MBI

In [11]:
# Define wavelengths
wavelengths = np.array([760, 720, 670, 610])

# Read in CSV file
df = pd.read_csv("/home/rebeccaz/Data/collapsed/2023_09_14_Collapsed_Data.csv")

for i, wavelength in enumerate(wavelengths):
    print("Wavelenght: " + str(wavelength))
    # Filter the DataFrame for the specific wavelength
    df_wavelength = df[(df["OBS-MOD"] == "IPOL_MBI")]
    
    # Further filter for each camera from the already filtered DataFrame
    df_cam1_FL1 = df_wavelength[(df_wavelength["U_CAMERA"] == 1) & (df_wavelength["U_FLC"] == "A")]
    df_cam1_FL2 = df_wavelength[(df_wavelength["U_CAMERA"] == 1) & (df_wavelength["U_FLC"] == "B")]
    df_cam2_FL1 = df_wavelength[(df_wavelength["U_CAMERA"] == 2) & (df_wavelength["U_FLC"] == "A")]
    df_cam2_FL2 = df_wavelength[(df_wavelength["U_CAMERA"] == 2) & (df_wavelength["U_FLC"] == "B")]

    # Extracting fluxes for each camera & FLC state
    # FL1 = df_cam1_FL1["TOTAL_FLUX"].values.astype(float)
    # FL2 = df_cam1_FL2["TOTAL_FLUX"].values.astype(float)
    # FR1 = df_cam2_FL1["TOTAL_FLUX"].values.astype(float)
    # FR2 = df_cam1_FL2["TOTAL_FLUX"].values.astype(float)

    FL1 = df_cam1_FL1["TOTAL_FLUX"].apply(lambda x: np.array(ast.literal_eval(x)[i]))
    FL2 = df_cam1_FL2["TOTAL_FLUX"].apply(lambda x: np.array(ast.literal_eval(x)[i]))
    FR1 = df_cam2_FL1["TOTAL_FLUX"].apply(lambda x: np.array(ast.literal_eval(x)[i]))
    FR2 = df_cam2_FL2["TOTAL_FLUX"].apply(lambda x: np.array(ast.literal_eval(x)[i]))

    FL1 = FL1.values.astype(float)
    FL2 = FL2.values.astype(float)
    FR1 = FR1.values.astype(float)
    FR2 = FR2.values.astype(float)

    # print("FL1: " + str(FL1))

    N = min(len(FL1), len(FL2), len(FR1), len(FR2))

    FL1_std = FL1.std()
    FL2_std = FL2.std()
    FR1_std = FR1.std()
    FR2_std = FR2.std()

    # print("FL1: " + str(FL1))
    # print("FL2: " + str(FL2))
    # print("FR1: " + str(FR1))
    # print("FR2: " + str(FR2))

    # Assuming em_gain.calculate_em_gain_ratio returns normalized_fluxes and em_gain_ratios
    normalized_fluxes, em_gain_ratios = \
        em_gain.calculate_em_gain_ratio(FL1, FL2, FR1, FR2)

    print("Number of Data Points: " + str(len(normalized_fluxes)))
    print(f"Median Normalized Flux for {wavelength}: {np.median(normalized_fluxes):.6f} ± {np.std(normalized_fluxes, ddof=1):.6f}")
    print(f"Median EM Gain Ratio for {wavelength}: {np.median(em_gain_ratios):.6f} ± {np.std(em_gain_ratios, ddof=1):.6f}")
    print()


Wavelenght: 760
Number of Data Points: 80
Median Normalized Flux for 760: -0.038372 ± 0.009963
Median EM Gain Ratio for 760: 1.079805 ± 0.021461

Wavelenght: 720
Number of Data Points: 80
Median Normalized Flux for 720: -0.089608 ± 0.030185
Median EM Gain Ratio for 720: 1.196856 ± 0.073416

Wavelenght: 670
Number of Data Points: 80
Median Normalized Flux for 670: -0.088673 ± 0.051897
Median EM Gain Ratio for 670: 1.194617 ± 0.125863

Wavelenght: 610
Number of Data Points: 80
Median Normalized Flux for 610: -0.103368 ± 0.056776
Median EM Gain Ratio for 610: 1.230574 ± 0.139922



# Examining 09/19/2025 MBI with error - Final EM gain standard deviation

In [6]:
# Define wavelengths
wavelengths = np.array([760, 720, 670, 610])

# Read in CSV file
df = pd.read_csv("/home/rebeccaz/Data/collapsed/2023_09_14_Collapsed_Data.csv")

for i, wavelength in enumerate(wavelengths):
    print("Wavelenght: " + str(wavelength))
    # Filter the DataFrame for the specific wavelength
    df_wavelength = df[(df["OBS-MOD"] == "IPOL_MBI")]
    
    # Further filter for each camera from the already filtered DataFrame
    df_cam1_FL1 = df_wavelength[(df_wavelength["U_CAMERA"] == 1) & (df_wavelength["U_FLC"] == "A")]
    df_cam1_FL2 = df_wavelength[(df_wavelength["U_CAMERA"] == 1) & (df_wavelength["U_FLC"] == "B")]
    df_cam2_FL1 = df_wavelength[(df_wavelength["U_CAMERA"] == 2) & (df_wavelength["U_FLC"] == "A")]
    df_cam2_FL2 = df_wavelength[(df_wavelength["U_CAMERA"] == 2) & (df_wavelength["U_FLC"] == "B")]

    # Extracting fluxes for each camera & FLC state
    # FL1 = df_cam1_FL1["TOTAL_FLUX"].values.astype(float)
    # FL2 = df_cam1_FL2["TOTAL_FLUX"].values.astype(float)
    # FR1 = df_cam2_FL1["TOTAL_FLUX"].values.astype(float)
    # FR2 = df_cam1_FL2["TOTAL_FLUX"].values.astype(float)

    FL1 = df_cam1_FL1["TOTAL_FLUX"].apply(lambda x: np.array(ast.literal_eval(x)[i]))
    FL2 = df_cam1_FL2["TOTAL_FLUX"].apply(lambda x: np.array(ast.literal_eval(x)[i]))
    FR1 = df_cam2_FL1["TOTAL_FLUX"].apply(lambda x: np.array(ast.literal_eval(x)[i]))
    FR2 = df_cam2_FL2["TOTAL_FLUX"].apply(lambda x: np.array(ast.literal_eval(x)[i]))

    FL1 = FL1.values.astype(float)
    FL2 = FL2.values.astype(float)
    FR1 = FR1.values.astype(float)
    FR2 = FR2.values.astype(float)

    # print("FL1: " + str(FL1))
    # print("FL2: " + str(FL2))
    # print("FR1: " + str(FR1))
    # print("FR2: " + str(FR2))

    # Assuming em_gain.calculate_em_gain_ratio returns normalized_fluxes and em_gain_ratios
    normalized_fluxes, em_gain_ratios = \
        em_gain.calculate_em_gain_ratio(FL1, FL2, FR1, FR2)

    print("Number of Data Points: " + str(len(normalized_fluxes)))
    print(f"Median Normalized Flux for {wavelength}: {np.median(normalized_fluxes):.6f} ± {np.std(normalized_fluxes, ddof=1):.6f}")
    print(f"Median EM Gain Ratio for {wavelength}: {np.median(em_gain_ratios):.6f} ± {np.std(em_gain_ratios, ddof=1):.6f}")
    print()


Wavelenght: 760
Number of Data Points: 80
Median Normalized Flux for 760: -0.038372 ± 0.009963
Median EM Gain Ratio for 760: 1.079805 ± 0.021461

Wavelenght: 720
Number of Data Points: 80
Median Normalized Flux for 720: -0.089608 ± 0.030185
Median EM Gain Ratio for 720: 1.196856 ± 0.073416

Wavelenght: 670
Number of Data Points: 80
Median Normalized Flux for 670: -0.088673 ± 0.051897
Median EM Gain Ratio for 670: 1.194617 ± 0.125863

Wavelenght: 610
Number of Data Points: 80
Median Normalized Flux for 610: -0.103368 ± 0.056776
Median EM Gain Ratio for 610: 1.230574 ± 0.139922



# Examine 09/19/2023 Data - Broadband

In [7]:
# Define wavelengths
wavelengths = np.array(["Open"])

# Read in CSV file
df = pd.read_csv("/home/rebeccaz/Data/20230919_VAMPIRES_Internal_Cal/collapsed/2023_09_19_Collapsed_Data.csv")

for i, wavelength in enumerate(wavelengths):
    print("Wavelenght: " + str(wavelength))
    # Filter the DataFrame for the specific wavelength
    df_wavelength = df[(df["U_FILTER"] == wavelength) & (df["OBS-MOD"] != "IPOL_MBI")]
    
    # Further filter for each camera from the already filtered DataFrame
    df_cam1_FL1 = df_wavelength[(df_wavelength["U_CAMERA"] == 1) & (df_wavelength["U_FLC"] == "A")]
    df_cam1_FL2 = df_wavelength[(df_wavelength["U_CAMERA"] == 1) & (df_wavelength["U_FLC"] == "B")]
    df_cam2_FL1 = df_wavelength[(df_wavelength["U_CAMERA"] == 2) & (df_wavelength["U_FLC"] == "A")]
    df_cam2_FL2 = df_wavelength[(df_wavelength["U_CAMERA"] == 2) & (df_wavelength["U_FLC"] == "B")]

    # Extracting fluxes for each camera & FLC state
    FL1 = df_cam1_FL1["TOTAL_FLUX"].values.astype(float)
    FL2 = df_cam1_FL2["TOTAL_FLUX"].values.astype(float)
    FR1 = df_cam2_FL1["TOTAL_FLUX"].values.astype(float)
    FR2 = df_cam1_FL2["TOTAL_FLUX"].values.astype(float)

    # print("FL1: " + str(FL1))
    # print("FL2: " + str(FL2))
    # print("FR1: " + str(FR1))
    # print("FR2: " + str(FR2))

    # Assuming em_gain.calculate_em_gain_ratio returns normalized_fluxes and em_gain_ratios
    normalized_fluxes, em_gain_ratios = \
        em_gain.calculate_em_gain_ratio(FL1, FL2, FR1, FR2)

    print("Number of Data Points: " + str(len(normalized_fluxes)))
    print(f"Median Normalized Flux for {wavelength}: {np.median(normalized_fluxes)}")
    print(f"Median EM Gain Ratio for {wavelength}: {np.median(em_gain_ratios)}")
    print()


Wavelenght: Open
Number of Data Points: 320
Median Normalized Flux for Open: -0.03420378536981787
Median EM Gain Ratio for Open: 1.070838646077178



# Examine 01/28/2022 Standards

In [8]:
# Performing for just unpolarized standards first
all_normalized_fluxes = []
all_em_gain_ratios = []

print("Examining Unpolarized Standards")
unpol_csv_directory = "../../../vampires_on_sky_calibration/data/unpolarized/csv"
unpol_csv_files = funcs.load_all_files_from_directory(unpol_csv_directory, ".csv")

for csv_file in unpol_csv_files:
    normalized_fluxes, em_gain_ratios = em_gain.process_vampires_dpp_csv_file(csv_file)
    all_normalized_fluxes = np.concatenate((all_normalized_fluxes, normalized_fluxes))
    all_em_gain_ratios = np.concatenate((all_em_gain_ratios, em_gain_ratios))

print()
print("Median Normalized Fluxes (Unpolarized): " + str(np.median(all_normalized_fluxes)))
print("Median EM Gain Ratio (Unnolarized): " + str(np.median(all_em_gain_ratios)))
print()

# Resetting to look at polarized standards
all_normalized_fluxes = []
all_em_gain_ratios = []

print("Examining Polarized Standards")
pol_csv_directory = "../../../vampires_on_sky_calibration/data/polarized/csv"
pol_csv_files = funcs.load_all_files_from_directory(pol_csv_directory, ".csv")

for csv_file in pol_csv_files:
    normalized_fluxes, em_gain_ratios = em_gain.process_vampires_dpp_csv_file(csv_file)
    all_normalized_fluxes = np.concatenate((all_normalized_fluxes, normalized_fluxes))
    all_em_gain_ratios = np.concatenate((all_em_gain_ratios, em_gain_ratios))

# Convert lists to numpy arrays if they are not already
all_normalized_fluxes = np.array(all_normalized_fluxes)
all_em_gain_ratios = np.array(all_em_gain_ratios)

# Filter out NaN values
valid_indices = ~np.isnan(all_normalized_fluxes) & ~np.isnan(all_em_gain_ratios)
all_normalized_fluxes = all_normalized_fluxes[valid_indices]
all_em_gain_ratios = all_em_gain_ratios[valid_indices]

print()
print("Median Normalized Fluxes (Polarized): " + str(np.median(all_normalized_fluxes)))
print("Median EM Gain Ratio (Polarized): " + str(np.median(all_em_gain_ratios)))


Examining Unpolarized Standards


FileNotFoundError: [Errno 2] No such file or directory: '../../../vampires_on_sky_calibration/data/unpolarized/csv'